In [1]:
################################################
#
# ptexture project
# 'PET texture analysis with Python'
#
#
# Kenji Hirata, MD, PhD
# Hokkaido University, Sapporo, Japan
# khirata@med.hokudai.ac.jp
# 1/1/2018
#
################################################

import numpy as np
import scipy.stats
import scipy.ndimage
import pandas as pd
from IPython.display import display
import os
import queue
import matplotlib.pyplot as plt
import datetime



################################################
#
# Report function
# Kenji Hirata, 1/1/2018
#
################################################

def do_not_report(obj):
    pass

def do_report(obj):
    if type(obj) == pd.core.frame.DataFrame:
        display(obj)
    else:
        print(obj)
    return

report = do_not_report


################################################
#
# Test data
# Kenji Hirata, 11/21/2017
#
################################################

def data1():
    data = np.array([
      3,3,4,4,5,
      3,3,4,3,5,
      7,3,4,3,5,
      3,3,4,4,5,
      3,3,4,4,5,

      6,2,2,4,5,
      2,2,4,4,9,
      2,7,9,9,4,
      1,2,4,9,2,
      2,1,4,2,9,

      3,3,4,4,5,
      3,9,4,4,5,
      3,3,9,4,5,
      3,8,4,4,5,
      3,8,4,4,5
    ])
    m = data.reshape((3,5,5))
    return m

def data2():
    data = np.array([
      3,3,4,4,5,7,3,
      3,3,4,3,5,5,3,
      7,3,4,3,5,4,4,
      3,3,4,4,5,4,4,
      3,3,4,4,5,6,1])
    m = data.reshape((5,7))
    return m
    
    
def data3():
    data = np.array([
        2,2,1,1,1,1,1,
        1,1,1,1,7,1,2,
        1,1,0,0,7,7,2,
        2,2,2,2,2,3,3,
        2,2,3,np.nan,3,4,4,
        5,5,5,3,3,3,3,
        3,3,3,3,3,2,3
    ])
    m = data.reshape((7,7))
    return m

def data4():
    data = np.array([
        1,1,1,1,7,
        1,0,0,1,7,
        1,1,2,2,np.nan,
        2,2,3,np.nan,3,
        5,5,5,3,3,
    ])
    m = data.reshape((5,5))
    return m

def data5():
    np.random.seed(123)
    data = np.random.rand(64)
    m = data.reshape((8,8))
    return m

def data6():
    df = pd.DataFrame({'x':[1,1,1,2,2,3,3,3], 'y':[1,2,3,1,2,1,2,3], 'v':[11,12,13,14,15,16,17,18]})
    return df

def data7():
    data = (np.arange(16))**2/10 + 5
    return data

def data8():
    df = pd.DataFrame({'x':[1,1,1,1,2,2,2,3,3,3,3,4,4,4,4], 'y':[1,2,3,4,1,2,3,1,2,3,4,1,2,3,4], 'v':[11,15,18,23,9,8,6,4,7,6,2,8,9,12,14]})
    return df

def data9():
    df = pd.DataFrame({'x':[1,1,1,1,2,2,2,3,3,3,3,4,4,4,4],
                       'y':[1,2,3,4,1,2,3,1,2,3,4,1,2,3,4],
                       'v':[11.5,15.7,18.9,23.2,9.4,8.7,6.1,4.8,7.1,6.2,2.9,8.7,9.5,12.4,14.3]})
    return df

def data10():
    df = pd.DataFrame({'x':[1,1,1,2,2,2,3,3],
                       'y':[1,2,3,1,2,3,1,2],
                       'v':[3.5, 5.5, 7.0, 3.0, 4.0, 2.0, 6, 5]})
    return df

def data11():
    nx, ny, nz = 5, 4, 3
    l = [(x,y,z) for z in range(nz) for y in range(ny) for x in range(nx)]
    df = pd.DataFrame(l, columns=['x','y','z'])
    df['v'] = df.index * 0.27
    df2 = df.drop([18,20,59])
    return df2


################################################
#
# generate_ndarray
# takes x,y,v DataFrame to generate numpy 2D array
# takes x,y,z,v DataFrame to generate numpy 3D array
#
# Kenji Hirata, 1/1/2018
#
################################################


def generate_2darray(df):
    minx = np.min(df.x)
    miny = np.min(df.y)
    maxx = np.max(df.x)
    maxy = np.max(df.y)
    m = np.empty((maxy - miny + 1, maxx - minx + 1))
    m.fill(np.nan)

    for i in range(len(df)):
        a = df.iloc[i]
        m[int(a.y)- miny, int(a.x)- minx] = a.v
    return m

def generate_3darray(df):
    minx = np.min(df.x)
    miny = np.min(df.y)
    minz = np.min(df.z)
    maxx = np.max(df.x)
    maxy = np.max(df.y)
    maxz = np.max(df.z)
    m = np.empty((maxz - minz + 1, maxy - miny + 1, maxx - minx + 1))
    m.fill(np.nan)

    for i in range(len(df)):
        a = df.iloc[i]
        m[int(a.z)- minz, int(a.y)- miny, int(a.x)- minx] = a.v
    return m

def generate_2darray_v1(df):
    report('Generating 2-dimensional array')
    minx = np.min(df.x)
    miny = np.min(df.y)
    maxx = np.max(df.x)
    maxy = np.max(df.y)
    m = np.empty((maxy - miny + 1, maxx - minx + 1))
    m.fill(np.nan)

    for i in range(len(df)):
        a = df.iloc[i]
        m[int(a.y)- miny, int(a.x)- minx] = a.v1
    return m

def generate_3darray_v1(df):
    report('Generating 3-dimensional array')
    minx = np.min(df.x)
    miny = np.min(df.y)
    minz = np.min(df.z)
    maxx = np.max(df.x)
    maxy = np.max(df.y)
    maxz = np.max(df.z)
    m = np.empty((maxz - minz + 1, maxy - miny + 1, maxx - minx + 1))
    m.fill(np.nan)

    for i in range(len(df)):
        a = df.iloc[i]
        m[int(a.z)- minz, int(a.y)- miny, int(a.x)- minx] = a.v1
    return m



################################################
#
# discritize
# takes ndarray of continuous value, returns same size of ndarray of nbin levels decrete value.
# The input ndarray may contain NaN.
# The result type is 'ndarray of float' rather than int because NaN does not exist in int type.
#
# Kenji Hirata, 11/21/2017
# modified on 12/04
#
################################################

def discritize(data, nbin, lo, hi, getCategories = False):
    '''If you want lo and hi to be min and max of the data, respectively,
    give np.nan for both lo and hi.
    '''
    if np.isnan(lo):
        lo = np.nanmin(data.ravel())
    if np.isnan(hi):
        hi = np.nanmax(data.ravel())
    
    if lo > hi:
        raise ValueError('lo must not be higher than hi.')
    
    R = np.floor((nbin * (data - lo) / (hi-lo)))
    R[R < 0] = 0
    R[R >= nbin] = nbin -1

    cats = np.array([(hi - lo) / nbin * i + lo for i in range(nbin+1)])
    report(('bins', nbin))
    report(('categories', cats))
    
    if getCategories:
        return R, cats
    else:
        return R 

    
################################################
#
# complement_crosstab
#
# Kenji Hirata, 11/21/2017
#
################################################

def complement_crosstab2(cr, index_hi=np.nan, column_hi=np.nan):
    if np.isnan(index_hi):
        index_hi = int(np.max(cr.index.values)+1)
    if np.isnan(column_hi):
        column_hi = int(np.max(cr.columns.values)+1)
    d = pd.DataFrame(np.zeros((index_hi, column_hi)), index=np.arange(index_hi), columns=np.arange(column_hi))
    cr1 = cr.add(d, fill_value=0)
    return cr1


################################################
#
# Usual parameters in Python
# Kenji Hirata, 11/22/2017
#
################################################

def usualParams(v):
    report('### Calculating usual parameters')
    
    SUVmax = max(v)
    SUVmean = np.mean(v)
    n = len(v) * 1*1*1
    s = np.sum(v)
    cols = ['SUVmax', 'SUVmean', 'NumOfVoxels', 'SUVsum']
    vals = [[SUVmax, SUVmean, n, s]]
    results = pd.DataFrame(data=vals, columns=cols)

    report(results)
    return results


################################################
#
# Usual parameters in Python
# Kenji Hirata, 11/22/2017
#
################################################

def histParams(hist):
    report('### Calculating histogram parameters')
    
    SDhist = np.std(hist, ddof=1)
    
    #Skewness = (1/N)*np.sum((h-np.mean(h))**3) / ((1/N)*np.sum((h-np.mean(h))**2))**(3/2)
    Skewness = scipy.stats.skew(hist)
    
    #Kurtosis = (1/N)*np.sum((h-np.mean(h))**4) / ((1/N)*np.sum((h-np.mean(h))**2))**2 - 3
    Kurtosis = scipy.stats.kurtosis(hist)
    
    tb = pd.crosstab(hist,0)
    p = complement_crosstab2(tb).values.ravel()/len(hist)
    #print('Probability of gray-level i', p)
    EnergyHist = np.sum(p*p)
    
    p_nz = p[p != 0]
    EntropyHist = -np.sum(p_nz*np.log(p_nz))
    
    cols = ['SDhist', 'Skewness', 'Kurtosis', 'EnergyHist', 'EntropyHist']
    vals = [[SDhist, Skewness, Kurtosis, EnergyHist, EntropyHist]]
    results = pd.DataFrame(data=vals, columns=cols)

    report(results)
    return results


################################################
#
# Gray-level co-occurrence matrix (GLCM) based texture parameters in Python
# Kenji Hirata, 1/2/2018
#
################################################


def comat2d(d, offset, nbin):
    d1 = pd.DataFrame({'v2': d.v1, 'y': d.y - offset[0],  'x': d.x + offset[1]})
    d2 = pd.merge(d,d1)
    cr = pd.crosstab(d2.v1, d2.v2)
    cr1 = complement_crosstab2(cr, nbin, nbin)
    return cr1, d2

def comat3d(d, offset, nbin):
    d1 = pd.DataFrame({'v2': d.v1, 'z': d.z - offset[0], 'y': d.y - offset[1], 'x': d.x - offset[2]})
    # subtracting offset may look tricky; with this operation current(v1) and (current + offset)(v2) are paired.
    d2 = pd.merge(d,d1)
    cr = pd.crosstab(d2.v1, d2.v2)
    cr1 = complement_crosstab2(cr, nbin, nbin)
    return cr1, d2

def comatParams2d(d, nbin):
    offsets = fourDirections()
    return comatParams(d,nbin,offsets)
    
def comatParams3d(d, nbin):
    offsets = thirteenDirections()
    return comatParams(d,nbin,offsets)

def fourDirections():
    return [(0,1), (1,0), (1,1), (1,-1)]

def thirteenDirections():
    return [(0,0,1), (0,1,1), (0,1,0), (0,1,-1),
            (1,0,0), (1,0,1), (1,1,1), (1,1,0),
            (1,1,-1), (1,0,-1),(1,-1,-1),(1,1,0),(1,-1,1)]

    # The total of 13 directions mean as follows:
    #
    # (0,0,1)  3 o'clock
    # (0,1,1)　4:30 o'clock
    # (0,1,0)　6 o'clock
    # (0,1,-1) 7:30 o'clock
    #  9  o'clock and later are omitted because it already appreared.
    # 
    #  Downward direction (delta z=1)
    # (1,0,0) center of the clock
    # (1,0,1) 3 o'clock
    # (1,1,1) 4:30 o'clock
    # (1,1,0) 6 o'clock
    # (1,1,-1) 7.5 o'clock
    # (1,0,-1) 9 o'clock
    # (1,-1,-1) 10:30 o'clock
    # (1,1,0) 12 o'clock
    # (1,-1,1) 1:30 o'clock
            
def comatParams(d, nbin, offsets):
    report('### Generating Co-occurence matrix')

    dim = len(offsets[0])
    seq = np.arange(nbin)
    ii = np.tile(seq, (nbin,1))
    jj = ii.T
    lst =[]

    for o in offsets:

        if dim==2:
            com, d2 = comat2d(d, o, nbin)
        elif dim==3:
            com, d2 = comat3d(d, o, nbin)
        
        report(('offset', o))
        report(com)
        cm = com.values
        
        cm = cm / np.sum(cm)
        cm_nz = cm[cm != 0]
        
        homogeneity = np.sum(cm / (1+np.abs(ii-jj)))
        energy = np.sum(cm**2)
        correlation = np.corrcoef(d2.v1, d2.v2)[0,1]
        
        contrast = np.sum((ii-jj)**2*cm)
        entropy = -np.sum(cm_nz * np.log(cm_nz))
        dissimilarity = np.sum(np.abs(ii-jj)*cm)

        ids = ['HomogeneityGLCM','EnergyGLCM','CorrelationGLCM','ContrastGLCM','EntropyGLCM','DissimilarityGLCM']
        se=pd.Series([homogeneity, energy, correlation, contrast, entropy, dissimilarity], ids)
        
        if dim==2:
            se.name = (o[0]+1, o[1]+1)
        elif dim==3:
            se.name = (o[0]+1, o[1]+1, o[2]+1)

        lst.append(se)
        
    df = pd.DataFrame(lst)
    ave = df.mean()
    ave.name = 'mean'
    df = df.append(ave)
    report(df)
    return df


################################################
#
# gray-level run-length matrix
# Kenji Hirata, 11/23/2017
#
################################################

def xline(x, m):
    ny,nx = m.shape
    return [(y,x) for y in range(ny)]

def yline(y, m):
    ny,nx = m.shape
    return [(y,x) for x in range(nx)]

def xplain(x, m):
    nz,ny,nx = m.shape
    return [(z,y,x) for z in range(nz) for y in range(ny)]

def yplain(y, m):
    nz,ny,nx = m.shape
    return [(z,y,x) for z in range(nz) for x in range(nx)]

def zplain(z, m):
    nz,ny,nx = m.shape
    return [(z,y,x) for y in range(ny) for x in range(nx)]


def rle(vec):
    if len(vec)==0:
        return [],[]
    if len(vec)==1:
        return vec,[1]
    
    prev = vec[0]
    count = 1
    vallst = []
    lenlst = []
    for a in vec[1:]:
        if (a == prev) or (np.isnan(a) and np.isnan(prev)):
            count += 1
        else:
            vallst.append(prev)
            lenlst.append(count)
            count = 1
            prev = a
    else:
        vallst.append(a)
        lenlst.append(count)
    return vallst,lenlst

def getGlrlm2d(discritizedArray, initialpoints, dy, dx):
    m = discritizedArray
    ny,nx = m.shape
    a1d =[]
    for current in initialpoints:
        cy,cx = current
        while((cy >= 0) and (cx >= 0) and (cy < ny) and (cx < nx)):
            a1d.append(m[cy,cx])
            cx += dx
            cy += dy
        a1d.append(np.nan)
    vals, lens = rle(a1d)
    
    flag = ~np.isnan(vals)
    vals1 = np.array(vals)[flag].astype(int)
    lens1 = np.array(lens)[flag]
    cr = pd.crosstab(vals1,lens1)
    cr1 = complement_crosstab2(cr)
    return cr1

def getGlrlm3d(discritizedArray, initialpoints, dz, dy, dx):
    m = discritizedArray
    nz,ny,nx = m.shape
    a1d =[]
    for current in initialpoints:
        cz,cy,cx = current
        while((cz >= 0) and (cy >= 0) and (cx >= 0) and (cz < nz) and (cy < ny) and (cx < nx)):
            a1d.append(m[cz,cy,cx])
            cx += dx
            cy += dy
            cz += dz
        a1d.append(np.nan)
    vals, lens = rle(a1d)
    
    flag = ~np.isnan(vals)
    vals1 = np.array(vals)[flag].astype(int)
    lens1 = np.array(lens)[flag]
    cr = pd.crosstab(vals1,lens1)
    cr1 = complement_crosstab2(cr)
    return cr1


def calcGlrlmParams(glrlm_or_glszm, mode):
    '''mode is either 'R' or 'Z'
    'R' indicates GLRLM, and 'Z' indicates GLSZM.
    The calculation formulas are completely same between 'R' and 'Z';
    the only difference is variable name (i.e., index of the returning pd.Series)
    '''
    
    # left-most column indicating number of run(size)=0 must be removed because it is unnecessary and even causes an adverse effect of divided-by-zero error.
    mat = glrlm_or_glszm.values[:,1:]

    sh = mat.shape
    jj = np.tile(np.arange(sh[1])+1, (sh[0],1))
    ii = np.tile(np.arange(sh[0])+1, (sh[1],1)).T
    
    SRE   = (1/np.sum(mat)) * np.sum(mat/jj**2)
    LRE   = (1/np.sum(mat)) * np.sum(mat*jj**2)
    LGRE  = (1/np.sum(mat)) * np.sum(mat/ii**2)
    HGRE  = (1/np.sum(mat)) * np.sum(mat*ii**2)
    SRLGE = (1/np.sum(mat)) * np.sum(mat/ii**2/jj**2)
    SRHGE = (1/np.sum(mat)) * np.sum(mat*ii**2/jj**2)
    LRLGE = (1/np.sum(mat)) * np.sum(mat/ii**2*jj**2)
    LRHGE = (1/np.sum(mat)) * np.sum(mat*ii**2*jj**2)
    GLNUr = (1/np.sum(mat)) * np.sum(np.sum(mat, axis=1)**2)
    RLNU  = (1/np.sum(mat)) * np.sum(np.sum(mat, axis=0)**2)
    RP    = np.sum(mat) / np.sum(jj * mat)

    if mode == 'R':
        ids = ['SRE', 'LRE', 'LGRE', 'HGRE', 'SRLGE', 'SRHGE', 'LRLGE', 'LRHGE', 'GLNUr', 'RLNU', 'RP']
    if mode == 'Z':
        ids = ['SZE', 'LZE', 'LGZE', 'HGZE', 'SZLGE', 'SZHGE', 'LZLGE', 'LZHGE', 'GLNUz', 'ZLNU', 'ZP']

    se = pd.Series([SRE, LRE, LGRE, HGRE, SRLGE, SRHGE, LRLGE, LRHGE, GLNUr, RLNU, RP], ids)
    return se


        
def combineGlrlm2d(discritizedArray):
    
    m = discritizedArray
    ny,nx = m.shape
    lst =[]

    def sub(m, points, dy, dx):
        g = getGlrlm2d(m, points, dy, dx)
        report(g)
        se = calcGlrlmParams(g, 'R')
        se.name = str((dy+1,dx+1))
        lst.append(se)
    
    points = xline(0, m)
    sub(m, points, 0, 1)

    points = yline(0, m)
    sub(m, points, 1, 0)

    points = set(xline(0, m)) | set(yline(0, m))
    sub(m, points, 1, 1)

    points = set(xline(0, m)) | set(yline(nx-1, m))
    sub(m, points, -1, 1)
    
    df = pd.DataFrame(lst)
    ave = df.mean()
    ave.name = 'mean'
    df = df.append(ave)
    report(df)
    return df

def combineGlrlm3d(discritizedArray):
    
    m = discritizedArray
    nz,ny,nx = m.shape
    lst =[]
    
    def sub(m, points, dz, dy, dx):
        g = getGlrlm3d(m, points, dz, dy, dx)
        report(g)
        se = calcGlrlmParams(g, 'R')
        se.name = str((dz+1,dy+1,dx+1))
        lst.append(se)
        
    ###
    points = xplain(0, m)
    sub(m, points, 0, 0, 1)
    
    points = yplain(0, m)
    sub(m, points, 0, 1, 0)
    
    points = zplain(0, m)
    sub(m, points, 1, 0, 0)
    ###
    
    ###
    points = set(xplain(0, m)) | set(yplain(0,m))
    sub(m, points, 0, 1, 1)

    points = set(yplain(0, m)) | set(zplain(0,m))
    sub(m, points, 1, 1, 0)

    points = set(zplain(0, m)) | set(xplain(0,m))
    sub(m, points, 1, 0, 1)

    points = set(xplain(0, m)) | set(yplain(ny-1,m))
    sub(m, points, 0, -1, 1)

    points = set(yplain(0, m)) | set(zplain(nz-1,m))
    sub(m, points, -1, 1, 0)

    points = set(zplain(0, m)) | set(xplain(nx-1,m))
    sub(m, points, 1, 0, -1)
    ###

    ###
    points = set(xplain(0, m)) | set(yplain(0,m)) | set(zplain(0,m))
    sub(m, points, 1, 1, 1)

    points = set(xplain(0, m)) | set(yplain(0,m)) | set(zplain(nz-1,m))
    sub(m, points, -1, 1, 1)

    points = set(xplain(0, m)) | set(yplain(ny-1,m)) | set(zplain(0,m))
    sub(m, points, 1, -1, 1)

    points = set(xplain(0, m)) | set(yplain(0,m)) | set(zplain(nz-1,m))
    sub(m, points, -1, 1, 1)
    ###

    df = pd.DataFrame(lst)
    ave = df.mean()
    ave.name = 'mean'
    df = df.append(ave)
    return df

def combineGlrlm(discritizedArray):
    
    report('### Generating GLRLM')
    
    if len(discritizedArray.shape) == 2:
        df = combineGlrlm2d(discritizedArray)
    elif len(discritizedArray.shape) == 3:
        df = combineGlrlm3d(discritizedArray)
    else:
        raise ValueError('discritizedArray must be 2- or 3-dimensional.')

    report(df)
    return df


################################################
#
# gray-level zone-size matrix in Python
# Kenji Hirata, 11/24/2017
#
################################################
    
def getGlzsm(discritizedArray, connection):
    m = discritizedArray
    if len(m.shape) == 2:
        if connection == 4:
            s = [[0,1,0],[1,1,1],[0,1,0]]
        elif connection == 8:
            s = [[1]*3]*3
        else:
            raise ValueError('connection must be 4 or 8 in case of 2-dimensional.')
    elif len(m.shape) == 3:
        if connection == 6:
            s = [[[0,0,0], [0,1,0], [0,0,0]],
                 [[0,1,0], [1,1,1], [0,1,0]],
                 [[0,0,0], [0,1,0], [0,0,0]]]
        elif connection == 18:
            s = [[[0,1,0], [1,1,1], [0,1,0]],
                 [[1,1,1], [1,1,1], [1,1,1]],
                 [[0,1,0], [1,1,1], [0,1,0]]]
        elif connection == 26:
            s = [[[1]*3]*3]*3
        else:
            raise ValueError('connection must be 6 or 18 or 26 in case of 3-dimensional.')
    else:
        raise ValueError('discritizedArray must be 2- or 3-dimensional.')

    lst = []
    maxgl = np.nanmax(m).astype(int)
    maxsz = 0
    for i in range(maxgl+1):#[2,3]:

        m1 = m==i
        a,_ = scipy.ndimage.label(m1, s)
        if np.sum(a)==0:
            continue
        b=pd.crosstab(a.flatten(), 0)
        c = pd.crosstab(b[1:].values.flatten(), 0)
        c.columns = [[i]]
        c = c.T

        maxsz1 = np.max(c.columns.values)
        if maxsz < maxsz1:
            maxsz = maxsz1

        lst.append(c)
    
    df = pd.DataFrame(np.zeros((maxgl+1,maxsz+1)), index = np.arange(maxgl+1), columns=np.arange(maxsz+1))
    for l in lst:
        df = df.add(l, fill_value =0)
    df = df.astype(int)
    return df

def combineGlzsm(discritizedArray, connection):
    report('### Generating GLZSM')

    m = discritizedArray
    df = getGlzsm(m, connection)
    report(df)
    ps = calcGlrlmParams(df, 'Z')
    res = pd.DataFrame([ps])
    report(res)
    return res


################################################
#
# neighborhood gray-level different matrix in Python
# Kenji Hirata, 11/27/2017
#
################################################

def getNgldm(discritizedArray, mode):
    m = discritizedArray
    if len(m.shape) == 2:
        return getNgldm2d(discritizedArray, mode)
    elif len(m.shape) == 3:
        return getNgldm3d(discritizedArray, mode)
    else:
        raise ValueError('discritizedArray must be 2- or 3-dimensional.')

def getNgldm2d(discritizedArray, mode):
    m = discritizedArray
    ny,nx = m.shape
    
    #fill periphery with numpy.nan
    nx+=2
    ny+=2
    m1 = np.empty((ny,nx))
    m1.fill(np.nan)
    m1[1:-1,1:-1] = m
    m = m1

    se = pd.Series()
    for i in range(int(np.nanmax(m)+1)):
        #print('i =',i)
        avelist = []
        for y in range(1,ny-1):
            for x in range(1,nx-1):
                if np.isnan(m[y,x]) == False:
                    if m[y,x] == i:
                        neighbors = [m[y-1,x-1], m[y-1,x  ], m[y-1,x+1],
                                     m[y  ,x-1],             m[y  ,x+1],
                                     m[y+1,x-1], m[y+1,x  ], m[y+1,x+1]]
                        if mode == 1:
                            ave = np.nanmean(neighbors)
                        elif mode == 2:
                            ave = np.mean(neighbors)
                        #print(x,y,'ave =',ave)
                        avelist.append(np.abs(i - ave))
        sum = np.nansum(avelist) # not mean but sum
        se1 = pd.Series({i:sum})
        se = se.append(se1)
    return se

def getNgldm3d(discritizedArray, mode):
    m = discritizedArray
    nz,ny,nx = m.shape
    
    #fill periphery with numpy.nan
    nz+=2
    ny+=2
    nx+=2    
    m1 = np.empty((nz,ny,nx))
    m1.fill(np.nan)
    m1[1:-1,1:-1,1:-1] = m
    m = m1

    se = pd.Series()
    for i in range(int(np.nanmax(m)+1)):
        avelist = []
        for z in range(1,nz-1):
            for y in range(1,ny-1):
                for x in range(1,nx-1):
                    if np.isnan(m[z,y,x]) == False:
                        if m[z,y,x] == i:
                            neighbors = [m[z-1,y-1,x-1], m[z-1,y-1,x  ], m[z-1,y-1,x+1],
                                         m[z-1,y  ,x-1], m[z-1,y  ,x  ], m[z-1,y  ,x+1],
                                         m[z-1,y+1,x-1], m[z-1,y+1,x  ], m[z-1,y+1,x+1],
                                         m[z  ,y-1,x-1], m[z  ,y-1,x  ], m[z  ,y-1,x+1],
                                         m[z  ,y  ,x-1],                 m[z  ,y  ,x+1],
                                         m[z  ,y+1,x-1], m[z  ,y+1,x  ], m[z  ,y+1,x+1],
                                         m[z+1,y-1,x-1], m[z+1,y-1,x  ], m[z+1,y-1,x+1],
                                         m[z+1,y  ,x-1], m[z+1,y  ,x  ], m[z+1,y  ,x+1],
                                         m[z+1,y+1,x-1], m[z+1,y+1,x  ], m[z+1,y+1,x+1]]
                            if mode == 1:
                                ave = np.nanmean(neighbors)
                            elif mode == 2:
                                ave = np.mean(neighbors)
                            avelist.append(np.abs(i - ave))
        sum = np.nansum(avelist) # not mean but sum
        se1 = pd.Series({i:sum})
        se = se.append(se1)
    return se


def calcNgldmParams(discritizedArray, ngldm, nbin):
    
    p1 = pd.crosstab(discritizedArray.ravel(),0)
    p = complement_crosstab2(p1, nbin).values.ravel()
    N = np.sum(p)
    p = p/N
    
    s = complement_crosstab2(pd.DataFrame(ngldm), nbin).values.ravel()

    jj = np.tile(np.arange(nbin),(nbin,1))
    ii = jj.T

    report('NGLDM')
    report(s)
    report('probability')
    report(p)

    Coarseness = 1 / (1e-20 + np.sum(p*s))

    Contrast = np.sum(np.outer(p,p) * (ii-jj)**2) * np.sum(s) / (N*nbin*(nbin-1))
    
    #busyness
    total = 0
    for i in range(len(p)):
        for j in range(len(p)):
            if not(p[i]==0) and not(p[j]==0):
                total += np.abs(i * p[i] - j * p[j])
    Busyness = np.sum(p*s) / total

    se = pd.Series([Coarseness,Contrast,Busyness], ['CoarsenessNGLDM','ContrastNGLDM','BusynessNGLDM'])
    return pd.DataFrame([se])

def combineNgldm(mat, nbin, mode):
    report('### Generating NGLDM')
    ngldm = getNgldm(mat, mode)
    res = calcNgldmParams(mat, ngldm, nbin)
    report(res)
    return res


################################################
#
# Get all params
# Takes DataFrame with columns of x,y,v --- v is continuous
#       nbin = 16, 32, 64, etc
# Returns DataFrame
#
# Kenji Hirata, 11/27/2017
#
################################################

def getAllParams2d(df, nbin, lo, hi, connection, ngldm_mode):
    
    results = []
    
    results.append(usualParams(df.v))
    
    df['v1'] = discritize(df.v, nbin, lo, hi)
    results.append(histParams(df.v1))
    
    results.append(comatParams2d(df, nbin)) # comatParams requires v1 columns, which is discreted version of v.
    
    mat = generate_2darray_v1(df)
    results.append(combineGlrlm(mat))
    
    results.append(combineGlzsm(mat, connection))
    
    results.append(combineNgldm(mat, nbin, ngldm_mode))
    
    return results

def getAllParams3d(df, nbin, lo, hi, connection, ngldm_mode):
    
    results = []
    
    results.append(usualParams(df.v))
    
    df['v1'] = discritize(df.v, nbin, lo, hi)
    results.append(histParams(df.v1))
    
    results.append(comatParams3d(df, nbin)) # comatParams requires v1 columns, which is discreted version of v.
    
    mat = generate_3darray_v1(df)
               
    results.append(combineGlrlm(mat))
    
    results.append(combineGlzsm(mat, connection))

    results.append(combineNgldm(mat, nbin, ngldm_mode))
    
    return results



################################################
#
# Convert ugly table to pretty table
# Takes list of DataFrame
# Returns DataFrame
#
# Kenji Hirata, 1/2/2018
#
################################################


def convertToPrettyTable(res):
    res1 = []
    for r in res:
        if len(r)==1:
            res1.append(r)
        else:
            s = r.iloc[-1,:]
            d = pd.DataFrame([s], index=[0])
            res1.append(d)
    return pd.concat(res1, axis=1)

def convertToPrettyTable2(res):
    res1 = []
    for r in res:
        if len(r)==1:
            res1.append(r)
        else:
            res1.append(stack1(r))
    return pd.concat(res1, axis=1)


def stack1(df):
    b=df.stack()

    def f(s):
        return str(s).replace('(','').replace(')','').replace(' ','').replace(',','')

    return pd.DataFrame([b.values], columns= [ind[1]+f(ind[0]) for ind in b.index])


###################################################
#
# Super batch for texture analyses in Python
# Kenji Hirata, 1/2/2018
#
###################################################


def superBatch(di, dim, nbin, lo, hi, connection, ngldm_mode, full = False):

    filenames = os.listdir(di)

    print("Files:", filenames)
    lst = []
    for f in filenames:
        print(str(datetime.datetime.now()), f)
        
        fullname = os.path.join(di,f)
        
        df = pd.read_csv(fullname, delimiter='\t')
        
        
        ##### temporary code
        ###df.columns = ['x','y','z','v']
        #####

        if dim==2:
            results = getAllParams2d(df, nbin, lo, hi, connection, ngldm_mode)
        elif dim==3:
            results = getAllParams3d(df, nbin, lo, hi, connection, ngldm_mode)
        else:
            raise ValueError('dim (dimension) must be 2 or 3.')
            
        if full:
            results_pretty = convertToPrettyTable2(results)
        else:
            results_pretty = convertToPrettyTable(results)
        
        #print(results_pretty.T)
        
        results_pretty.index = [f]
        
        lst.append(results_pretty)

    results_final = pd.concat(lst)
    return results_final



###################################################
#
# Super batch 2 for texture analyses in Python
# Kenji Hirata, 12/22/2017
#
###################################################


def superBatch2(di, dim, nbin, lo, hi, connection, ngldm_mode):

    filenames = os.listdir(di)

    print("Files:", filenames)
    lst = []
    for f in filenames:
        print(str(datetime.datetime.now()))
        print(f)
        
        fullname = os.path.join(di,f)
        
        df = pd.read_csv(fullname, delimiter='\t')
        
        
        ##### temporary code
        
        df.rename(columns = {'v1':'v'}, inplace=True)
        df = df[['x','y','z','v']]
        #print(df)
        
        #df.columns = ['roinum','x','y','z','v','fmiso','gd','flair']
        #####

        if dim==2:
            results = getAllParams2d(df, nbin, lo, hi, connection, ngldm_mode)
        elif dim==3:
            results = getAllParams3d(df, nbin, lo, hi, connection, ngldm_mode)
        else:
            raise ValueError('dim (dimension) must be 2 or 3.')
            
        results_pretty = convertToPrettyTable(results)
        print(results_pretty.T)
        
        results_pretty.index = [f]
        
        lst.append(results_pretty)

    results_final = pd.concat(lst)
    return results_final

print('OK')


OK


In [ ]:
# Template for Running ptexture


# This indicates the directory where you have file(s) of x,y,z,v text.
di = 'testdata'

# Run
# dim (dimention) is 2 or 3. 2 for polar map; 3 for volume data.
# nbin is number of discritization bin, usually 64
# lo and hi specifies lower and upper limit used for discritization
#   if lo = np.nan and hi = np.nan, the global min and max are used for lo and hi.
# connection must be 2 or 4 for 2-d, and 6, 18, or 26 for 3-d (detail will be explained later)
# ngldm_mode = 1 uses np.nanmean(), mode == 2 uses np.mean() to calculate NGLDM
df = superBatch(di, dim = 3, nbin = 64, lo = 0, hi = 20, connection = 26, ngldm_mode = 1)
df.to_excel('result_file_name.xlsx')

# min-max case
#df = superBatch(di, dim = 3, nbin = 64, lo = np.nan, hi = np.nan, connection = 26, ngldm_mode = 1)
#df.to_excel('result_file_name_minmax.xlsx')
